In [3]:
import pandas as pd

def process_csv(input_csv: str, output_csv: str, drop_analysis: bool = True):
    """
    1. 给 A 列统一加前缀“选”
    2. 将 analysis 列合并到 A 列中
    
    :param input_csv: 输入 CSV 文件路径
    :param output_csv: 输出 CSV 文件路径
    :param drop_analysis: 是否删除 analysis 列
    """
    # 读取 CSV
    df = pd.read_csv(input_csv, encoding="gbk")  # 关键修复点

    # 1. A 列统一加前缀“选”
    df["A"] = df["A"].astype(str).apply(lambda x: f"选{x}")

    # 2. 合并 analysis 列到 A 列
    df["A"] = df["A"] + "\n\n" + df["analysis"].fillna("")

    # 可选：删除 analysis 列
    if drop_analysis:
        df.drop(columns=["analysis"], inplace=True)

    # 保存结果
    df.to_csv(output_csv, index=False, encoding="utf-8-sig")


In [4]:
process_csv("通信原理多选题.csv", "通信原理多选题2.csv")

In [16]:
import pandas as pd
import csv
import re


def flatten_to_one_line(text: str) -> str:
    """
    将多行文本压缩为单行：
    - 去除所有换行符
    - 合并多余空白
    """
    if pd.isna(text):
        return ""
    text = str(text)
    # 去除换行
    text = text.replace("\r\n", " ").replace("\r", " ").replace("\n", " ")
    # 合并多空格
    text = re.sub(r"\s+", " ", text)
    return text.strip()


def csv_q_a_single_line(
    input_csv: str,
    output_csv: str,
    encoding_in: str = "utf-8-sig",   # 如果报错可改成 "gbk"
    encoding_out: str = "utf-8-sig",
):
    # 读取 CSV（对多行字段更稳）
    df = pd.read_csv(
        input_csv,
        encoding=encoding_in,
        engine="python",
        sep=",",
        quotechar='"',
        quoting=csv.QUOTE_MINIMAL,
        keep_default_na=False,
    )

    # 清理列名
    df.columns = [c.strip() for c in df.columns]

    # 压缩 Q / A 为单行
    df["Q"] = df["Q"].apply(flatten_to_one_line)
    df["A"] = df["A"].apply(flatten_to_one_line)

    # 写出 CSV（注意：参数名是 lineterminator）
    df.to_csv(
        output_csv,
        index=False,
        encoding=encoding_out,
        quoting=csv.QUOTE_ALL,   # 强制所有字段加引号，最大兼容性
        lineterminator="\n",
    )


In [17]:
csv_q_a_single_line("通信原理多选题.csv", "通信原理多选题2.csv")